In [21]:
import pandas as pd
import numpy as np
import random
import math
# import matplotlib.pyplot as plt

df=pd.read_csv("Project3.csv") # Importing the data in dataframe
df.drop(df.columns[[0]],1,inplace=True) # Dropping the column named ID from the dataframe
# print(df)

In [22]:
# In order to calculate z-score, we need to first calculate the mean and standard deviation
# z-score is calculated to normalize the parameters so that every parameter has equal contribution

# Now Here we will calculate the mean of the data
rows=df.shape[0]
cols=df.shape[1]

avg_arr = [0]*cols
for y in range(0,cols):
  sum=0
  for x in range(0,rows):
    sum=sum+df.iloc[x,y]
  sum=sum/rows
  avg_arr[y]=sum
# avg_arr will have the mean of all the columns stored in it

In [23]:
# Now we will calculate the standard deviation of the data
std_dev = [0]*cols
for y in range(0,cols):
  sum=0
  for x in range(0,rows):
    diff=df.iloc[x,y]-avg_arr[y]
    diff=diff**2
    sum=sum+diff
  sum=sum/rows
  sum=math.sqrt(sum)
  std_dev[y]=sum
# std_dev will have the standard deviation of all the columns stored in it

In [24]:
# Now we will iterate over the dataframe and change every value to its z-score
for x in range(rows):
  for y in range(cols):
    df.iloc[x,y]=(df.iloc[x,y]-avg_arr[y])/std_dev[y] # This is the formula to get the z-score

# print(df)

In [25]:
# dist function takes 3 parameters namely idx (which is the row number - 0 indexed), k (the number of clusters), and
# the centroids (which is a 2d array and stores the data of centroids of every cluster)

# Let's suppose the value of k is 3, then centroids will have 3 rows and columns equal to the columns in dataframe
# To sum up, each row will have all the features of the ith centroid

def dist(idx,k,centroids):
  container=0
  mindist=math.inf
  for i in range(k):
    sum=0
    for j in range(cols):
      sum=sum+((centroids[i][j]-df.iloc[idx,j])**2)
    if sum<mindist: 
      mindist=sum
      container=i
  return container
# This function compares the distance of the point with each centroid and returns the container number whose centroid has the 
# closest distance from the point

In [26]:
# Elbow method is used to get the optimal number of clusters required for the data
# This Elbow method has been used two ways, one for returning the wss score when flag = 0 and the other for returning the cluster when flag = 1 
def elbow(k,flag):
  random_rows=random.sample(range(0,rows),k) # Select k random points and put them in random_rows array
  centroids=np.zeros((k,cols)) # Initialized the centroids with all zeros
  new_centroids=np.zeros((k,cols))

  for i in range(k):
    centroids[i]=df.iloc[random_rows[i],:] # Put the features of those random selected points in the centroids 2D array
    # Now the centroids 2D array contains randomly selected k rows of the dataframe
  
  # We will perform at max 30 iterations to get the convergerd centroids of the clusters 
  itr=0
  while(itr<30):
    itr=itr+1

    clusters=[[] for _ in range(k)] # Need to make clusters to store points in the clusters
    
    # This for loop puts all the points of the dataframe in the appropriate cluster
    for idx in range(rows):
      clusters[dist(idx,k,centroids)].append(idx)

    for cluster_idx, cluster in enumerate(clusters):
      cluster_mean=np.mean(df.iloc[cluster],axis=0) # Mean of each feature of each cluster is calculated to 
      # get the new centroid
      new_centroids[cluster_idx]=cluster_mean # The data is put in the new_centroids 2D array

    # Condition to check the convergence of the centroids
    cnt=0
    for i in range(k):
      for j in range(cols):
        if abs(new_centroids[i][j]-centroids[i][j]) <= 1e-5: 
          cnt=cnt+1

    if cnt==(k*cols) or itr==30:
      # If the centroids converged then this condition has to be followed
      score=0
      for i in range(k):
        for val in clusters[i]:
          for j in range(cols):
            score=score+((centroids[i][j]-df.iloc[val,j])**2)
            
      if flag==0:
        return score
      else:
        return clusters
    else:
      # If the centroids have not converged till now then move to the next loop but before that update the centroid
      centroids=new_centroids.copy()  

In [27]:
# # Here we will apply Elbow method to get the optimal value of k
# wss=[0]*10 # wss array contains the within cluster sum of squared errors
# for k in range(1,11):
#   wss[k-1]=elbow(k,0)

In [28]:
# # Once the wss score has been calculated, we need to figure out where the elbow occured
# stslope=wss[0]-wss[1]
# for i in range(1,11):
#   slope=wss[i]-wss[i+1]
#   if stslope>=10*slope:
#     k=i+1
#     break
# print(k)

In [29]:
# Here is the optimized algorithm that can be used instead of above 12 lines and it works exactly the same and at the same time more efficient
# Here we will apply Elbow method to get the optimal value of k
# We consecutively will calculate the wss score and at the same time will figure out where the elbow occured
wss=[0]*10 # wss array contains the within cluster sum of squared errors
wss[0]=elbow(1,0)
wss[1]=elbow(2,0)

stslope=wss[0]-wss[1]
for i in range(3,11):
    wss[i-1]=elbow(i,0)
    slope=wss[i-2]-wss[i-1]
    if stslope>=5*slope:
        k=i-2
        break
print("The Optimal value of k came out to be: ",k)
# This method will save us a lot of computation hence saving a lot of time

The Optimal value of k came out to be:  4


In [30]:
# Once we got the optimal value of k (the number of clusters), now it's time to put the points in appropriate cluster
final_clusters=[[] for _ in range(k)]
final_clusters=elbow(k,1)

# Now we alloted the cluster to the rows
ans=[0]*2000
for i in range(k):
  for val in final_clusters[i]:
    ans[val]=i+1

# print(ans)
content=""
for i in ans:
  content=content+str(i)+" "

OutF=open("19IE10041_P3.out","w") 
OutF.write(content) # The results are printed in the file
OutF.close()